In [108]:

#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause

from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron,LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn.decomposition import PCA
from sklearn import metrics
import pandas as pd
TRAIN_FILEPATH = "../Translated/cleaned/train.csv"
TEST_FILEPATH = "../Translated/cleaned/test.csv"
SS_FILEPATH = "../data/SampleSubmission.csv"
VECTORS_FILEPATH = ""
train_df = pd.read_csv(TRAIN_FILEPATH)
test_df = pd.read_csv(TEST_FILEPATH)
from sklearn.model_selection import KFold,StratifiedKFold,cross_val_score,train_test_split,StratifiedShuffleSplit


In [140]:
categories = train_df.Label.unique().tolist()
# data_train, data_test = train_test_split(train_df, test_size=0.1, random_state = 42, stratify = train_df['Label'])
data_train = train_df.copy()
data_test = test_df.copy()
train_idx = data_train.index.tolist()
test_idx = data_test.index.tolist()
def vect(max_df = 0.5):
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=max_df, stop_words='english')
    X_train = vectorizer.fit_transform(data_train.Text)
    return vectorizer
# def preprocess(df
from sklearn.metrics import accuracy_score

def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    skf = StratifiedKFold(n_splits=5, random_state=0)
    models, preds, scores = [], [],[]
    vectorizer = vect(max_df = 0.5)
    for train, test in skf.split(data_train.Text, data_train.Label):
#     print(train, test)
#     clf = LogisticRegression(penalty='l1')
        clf.fit(vectorizer.transform(data_train.Text.loc[data_train.index.intersection(train)]), data_train.Label.loc[data_train.index.intersection(train)])
        preds.append(clf.predict_proba(vectorizer.transform(data_test.Text)))
        models.append(clf)
        scores.append(accuracy_score(clf.predict(vectorizer.transform(data_train.Text.loc[data_train.index.intersection(test)])),data_train.Label.loc[data_train.index.intersection(test)]))
#         coefs.append(clf.coef_[0])
#         clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
#     pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    pred = np.mean(preds,axis = 0)
#     score = metrics.accuracy_score(data_test.Label, pred)
#     print("accuracy:   %0.3f" % score)
    return models, pred,scores
    

In [148]:
models, pred, scores = benchmark(LogisticRegression(C = 20, multi_class = 'ovr' ,tol=1e-5,random_state = 0))

________________________________________________________________________________
Training: 
LogisticRegression(C=20, multi_class='ovr', random_state=0, tol=1e-05)


C:\Users\amakr\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


train time: 54.889s
test time:  0.000s


In [149]:
np.mean(scores)

0.6295392953929538

In [131]:
# skf = StratifiedKFold(n_splits=5, random_state=0)
# models, preds = [], []
# vectorizer = vect(max_df = 0.5)
# for train, test in skf.split(data_train.Text, data_train.Label):
#       print(test)
# data_train.Text.loc[]
# np.zeros(pred[1].shape)
data_test['Label'] = models[4].classes_[np.argmax(pred,axis =1)]
sub = data_test[['ID', 'Label']]
sub.to_csv('cross_val.csv', index = False)
# [np.argmax(pred,axis =1)]
# models[4].classes_ == models[3].classes_ 
# models[4].classes_[[1,3,3]]
# pred

In [132]:
# train.tolist()
np.mean(pred,axis = 0)

array([0.00396599, 0.01250032, 0.05772551, 0.02401101, 0.05205382,
       0.00395089, 0.09315155, 0.09812242, 0.01560303, 0.00713147,
       0.01533163, 0.21370458, 0.02732427, 0.10318962, 0.10077275,
       0.10486998, 0.02917927, 0.00565802, 0.02309053, 0.00866335])

In [68]:
# (pred[0][0,0] +pred[1][0,0]+pred[2][0,0]+pred[3][0,0]+pred[4][0,0])/5

0.004026890749123799